In [47]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import label_binarize
import datetime

In [2]:
df = pd.read_csv('../data/aden_golfe.csv',sep=';')

In [18]:
max_quinzaine=24
max_month=12
max_area_lat=12
max_area_lon=31
number_grid=(max_area_lat+1)*(max_area_lon+1)
print df.head()

   Annee (0)  jour (0)  mois (0)  dist cote (nq) zone lat (0)  long(0)  \
0       2010        28        10             NaN  BCD  12:08N  054:25E   
1       2010        30        10             NaN  BCD  15:06N  055:58E   
2       2010         1         1             NaN  BCD  12:31N  047:18E   
3       2010         3         2             NaN  BCD  13:04N  047:04E   
4       2010         5         2             NaN  BCD  13:00N  048:45E   

           type(0)           pavillon statut de l'attaque(0)  ...    \
0             Dhow         Not Stated               Steaming  ...     
1     Bulk Carrier              Malta               Steaming  ...     
2  Chemical Tanker          Singapore               Steaming  ...     
3    General Cargo        North Korea               Steaming  ...     
4     Bulk Carrier  Antigua & Barbuda               Steaming  ...     

  quinzaine_week                                      newDescriptif  Guns  \
0             20                           pirates 

In [4]:
df['lat_area'] = df['lat_area'].astype(int)
df['lon_area'] = df['lon_area'].astype(int)

# Version 1 : Création d'une matrice en regroupant par quinzaine et zone

In [5]:
df['area'] = df.apply(lambda row: str(row['lat_area'])+'_'+str(row['lon_area']), axis=1)

## Creation d'un data frame vide

In [52]:
dfAgregateTotal=pd.DataFrame({'count' : df.groupby( [ "Annee (0)","quinzaine_week","area"] ).size()}).reset_index()

In [53]:
specifecArea=pd.DataFrame(dfAgregateTotal['area'].str.split('_',1).tolist(),columns = ['lat_area','lon_area'])
dfAgregateTotal=pd.concat([dfAgregateTotal, specifecArea], axis=1)
print dfAgregateTotal.head()

   Annee (0)  quinzaine_week  area  count lat_area lon_area
0       2008               3  4_20      2        4       20
1       2008               6  3_21      1        3       21
2       2008               6  4_18      1        4       18
3       2008               6  6_19      1        6       19
4       2008               7  5_16      1        5       16


In [54]:
uniqueYears=dfAgregateTotal['Annee (0)'].unique()
Colyears=np.repeat(uniqueYears, 24*(max_area_lon+1)*(max_area_lat+1))

TwoweekRepeat=np.repeat(np.linspace(1, max_quinzaine, num=max_quinzaine), (max_area_lon+1)*(max_area_lat+1))
ColTwoweek=np.tile(TwoweekRepeat, len(uniqueYears))

latRepeat=np.linspace(0, max_area_lat,num=max_area_lat+1)
ColLatArea=np.repeat(latRepeat, max_area_lon+1)
ColLatArea=np.tile(ColLatArea, max_quinzaine)
ColLatArea=np.tile(ColLatArea, len(uniqueYears))

#print ColLatArea

lonRepeat=np.tile(np.linspace(0, max_area_lon, num=max_area_lon+1),max_area_lat+1)
ColLonArea=np.tile(lonRepeat, max_quinzaine)
ColLonArea=np.tile(ColLonArea, len(uniqueYears))

print ColLonArea

count=np.repeat(0, len(Colyears))

print len(Colyears)
print len(ColTwoweek)
print len(ColLatArea)
print len(ColLonArea)


[  0.   1.   2. ...,  29.  30.  31.]
69888
69888
69888
69888


## Fill the matrice

In [55]:
#columns = ['Years','2_week', 'lat_area','lon_area','count']
#matriceML=pd.DataFrame(columns=columns)
#matriceML = matriceML.fillna(0)

dic = {'Years': Colyears, 
     '2_week': ColTwoweek, 
     'lat_area': ColLatArea,
     'lon_area': ColLonArea,
     'count': count
      }

dfFinal= pd.DataFrame(dic)

dfFinal['lat_area']=dfFinal['lat_area'].astype(int).astype('str')
dfFinal['lon_area']=dfFinal['lon_area'].astype(int).astype('str')

print dfFinal.head()

   2_week  Years  count lat_area lon_area
0       1   2008      0        0        0
1       1   2008      0        0        1
2       1   2008      0        0        2
3       1   2008      0        0        3
4       1   2008      0        0        4


In [56]:
for index, row in dfAgregateTotal.iterrows():
    if index%100==0:
        print index
    #print row['Annee (0)'], row['quinzaine_week'], row['count'] ,  row['lat_area'] , row['lon_area'] 
    idx=dfFinal[(dfFinal['Years']==row['Annee (0)']) & (dfFinal['lat_area']==row['lat_area']) & (dfFinal['lon_area']==row['lon_area'] )& (dfFinal['2_week']==row['quinzaine_week'])]
    #print 'index : '+str(idx.index.values)
    #print 'count : '+str(row['count'])
    dfFinal.set_value(idx.index.values, 'count',row['count'])

0
100
200
300


In [38]:
dfFinal['count'].sum()

421

## Write matrice in a file

In [57]:
dfFinal.to_csv("../data/aden_golfe_matrix_area.csv",sep=';', encoding='utf-8', index=False)

#  

# Version 2 : Création d'une matrice en regroupant par mois

In [84]:
dfAgregateTotal2=pd.DataFrame({'count' : df.groupby( [ "Annee (0)","mois (0)"] ).size()}).reset_index()
dfAgregateTotal2.head()

,Annee (0),mois (0),count
0,2008,2,2
1,2008,3,3
2,2008,4,6
3,2008,5,6
4,2008,6,2


In [85]:
unique_years=dfAgregateTotal2['Annee (0)'].unique()
print unique_years

[2008 2009 2010 2011 2012 2013 2014]


In [86]:
col_years=np.repeat(unique_years, max_month)
print len(col_years)

two_week_repeat=np.linspace(1, max_month, num=max_month)
#print np.repeat(two_week_repeat,4)
col_two_week=np.tile(two_week_repeat, len(unique_years))
print len(col_two_week)

col_count=np.repeat(0, len(col_two_week))


dic2 = {'Years': col_years, 
     'Month': col_two_week, 
     'Count': col_count#,
     #'type_attack': col_type
       }

dfFinal_2 = pd.DataFrame(dic2)


84
84


## Matrix empty

### On créé tout d'abord une matrice vide

In [87]:
dfFinal_2.head()

,Count,Month,Years
0,0,1,2008
1,0,2,2008
2,0,3,2008
3,0,4,2008
4,0,5,2008


# Fill matrice

In [88]:
#
for index, row in dfAgregateTotal2.iterrows():

    #print 'Years '+str(row['Annee (0)']),'month : '+str( row['mois (0)']),'count : '+str(row['count'])#,'type_attack : '+str(row['type d\'attaque(0)'])

    idx=dfFinal_2[(dfFinal_2['Years']==row['Annee (0)']) & (dfFinal_2['Month']==row['mois (0)'])]
    #print 'index : '+str(idx.index.values)
    #print 'count : '+str(row['count'])
    dfFinal_2.set_value(idx.index.values, 'Count',row['count'])

dfFinal_2.head()

,Count,Month,Years
0,0,1,2008
1,2,2,2008
2,3,3,2008
3,6,4,2008
4,6,5,2008


In [89]:
def monthCorrect(month):
    month=str(int(month))
    if len(month)==1:
        return '0'+month
    else:
        return month
print monthCorrect(10)

10


In [90]:
type(int(dfFinal_2['Month'][0]))

int

In [91]:
monthCorrect(dfFinal_2['Month'][0])

'01'

In [92]:
dfFinal_2['date']=dfFinal_2.apply(lambda row: str(int(row['Years']))+'-'+monthCorrect(row['Month'])+'-01', axis=1)
dfFinal_2['date']=dfFinal_2.apply(lambda row: datetime.datetime.strptime(row['date'], "%Y-%m-%d"), axis=1)

#dfFinal_2['full_years'] =dfFinal_2.apply(lambda row: row['Years']+float(row['2_week']/24), axis=1)
print dfFinal_2.head()

   Count  Month  Years       date
0      0      1   2008 2008-01-01
1      2      2   2008 2008-02-01
2      3      3   2008 2008-03-01
3      6      4   2008 2008-04-01
4      6      5   2008 2008-05-01


In [93]:
dfFinal_2.to_csv("../data/aden_golfe_matrix_global.csv",sep=';', encoding='utf-8', index=False)